In [20]:
import dash
from dash import html, dcc, Input, Output, Dash
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from datetime import date

In [2]:
data = pd.read_csv("Coffee Shop Sales.csv")
data["transaction_date"] = pd.to_datetime(data["transaction_date"])
data

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01,7:06:11,2,5,Lower Manhattan,32,3.00,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01,7:08:56,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01,7:14:04,2,5,Lower Manhattan,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01,7:20:24,1,5,Lower Manhattan,22,2.00,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01,7:22:41,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
...,...,...,...,...,...,...,...,...,...,...,...
149111,149452,2023-06-30,20:18:41,2,8,Hell's Kitchen,44,2.50,Tea,Brewed herbal tea,Peppermint Rg
149112,149453,2023-06-30,20:25:10,2,8,Hell's Kitchen,49,3.00,Tea,Brewed Black tea,English Breakfast Lg
149113,149454,2023-06-30,20:31:34,1,8,Hell's Kitchen,45,3.00,Tea,Brewed herbal tea,Peppermint Lg
149114,149455,2023-06-30,20:57:19,1,8,Hell's Kitchen,40,3.75,Coffee,Barista Espresso,Cappuccino


'Astoria'

In [41]:
# Creating a unique list for the Locations
Store = data.store_location.unique()
Store.sort()

# Create a list for the mode
Mode = ["Revenue", "Quantity"]
# Dictionary interpreter for the mode
ModeMeaning = {"Revenue" : "unit_price", "Quantity" : "transaction_qty"}

App = Dash(__name__)

# Defining the layout
App.layout = (
    html.Div([
        # Header Text
        html.H1("Store sales", style = {"fontSize" : 64, "textAlign" : "center"}),
        # 1st Selectable: Location
        html.Div([
            html.Label("Select the location: ", style = {"fontSize" : 28}),
            dcc.Dropdown(id = "Location", options = Store, value = "Astoria", style = {"width" : "40%"})]),
        # 2nd Selectable: Date
        html.Div([
            dbc.Row([html.Label("Select date range: ", style = {"fontSize" : 28})]),
            dbc.Row([dcc.DatePickerRange(id = "Date", min_date_allowed = date(2023,1,1), max_date_allowed = date(2023,12,31), start_date = date(2023,1,1), end_date = date(2023,12,31))])
            ]),

        # 3rd Selectable: Mode (Revenue or Quantity Sold)
        html.Div([
            dbc.Row([html.Label("Select mode: ", style = {"fontSize" : 28})]),
            dbc.Row([dcc.RadioItems(id = "Mode", options = Mode, value = "Revenue", inline = True)])
            ]),
        # The Graph!!!
        dcc.Graph(id = "graph")
]))

# Defining callback
@App.callback(
    Output("graph","figure"),
    Input("Location", "value"),
    Input('Date', 'start_date'),
    Input('Date', 'end_date'),
    Input("Mode", "value"))

def GetFig(Location,start_date,end_date, Mode):
    #Load Data
    data = pd.read_csv("Coffee Shop Sales.csv")

    # Convert to date
    data["transaction_date"] = pd.to_datetime(data["transaction_date"])

    # Sort the table as it was previously selected
    data = data.query("store_location == @Location and transaction_date.between(@start_date, @end_date)")

    # Define the Graph
    Fig = px.histogram(data, x = "product_category"	, y = ModeMeaning[Mode],
                   title = "Sales",
                   text_auto = True,
                   labels = {"x" : "Category",
                             "y" : "Revenue"},
                   template = "plotly_dark",
                   color_discrete_sequence=['white'],
                   height = 400,
                   width = 1200)
    Fig.update_layout(xaxis_title = "Category",
                      yaxis_title = Mode)
    return Fig

App.run()

'unit_price'